# RK Interpolation

This document includes Python codes that conduct Regression Kriging (RK) Interpolation on each waterbody, including Guana Tolomato Matanzas (GTM), Estero Bay (EB), Charlotte Harbor (CH), Biscayne Bay (BB), Big Bend Seagrasses (BBS).
parameters, including Dissolved oxygen (DO_mgl), Salinity (Sal_ppt), Turbidity (Turb_ntu), Temperature (T_c), Secchi (Secc_m), Total Nitrogen (TN_mgl) in arcpy environment.

The analysis is conducted in the separate managed parameters of Total Nitrogen (TN_mgl), Dissolved oxygen (DO_mgl), Salinity (Sal_ppt), Turbidity (Turb_ntu), Temperature (T_c), and Secchi (Secc_m) in arcpy environment.

* [1. Data Preprocess](#preprocessing)
* [2. Generate Shapefiles](#create_shp)
* [3. Regression Kriging](#rk)

In [1]:
import pandas as pd
import numpy  as np
import arcpy
from arcpy.sa import *
import os
import math
import importlib
import sys
path = r'../git/misc'
sys.path.insert(0, path)
import RK
importlib.reload(RK)
import warnings
warnings.filterwarnings('ignore')

# 1. Preprocessing <a class="anchor" id="preprocessing"></a>

## If you are the first time to run this code, you could run the following cell. 
## If you have already generated the combining csv, you could skip this cell and just run the next cell.

In [2]:
# Define water quality files
gis_path = r'F:/SEACAR_WQ_2024/SEACAR_WQ_2024/GIS_Data/'


dfDis_orig = pd.read_csv(gis_path + "OEAT_Discrete_WQ-2024-Jan-16.csv", low_memory=False)
dfCon_orig = pd.read_csv(gis_path + "OEAT_Continuous_WQ-2024-Jan-16.csv", low_memory=False)

# Combine discrete and continuous file
dfAll_orig = pd.concat([dfDis_orig, dfCon_orig], ignore_index=True)
dfAll_orig.to_csv(gis_path + "OEAT_All_WQ-2023-Jan-16.csv", index=False)

In [2]:
gis_path = r'F:/SEACAR_WQ_2024/SEACAR_WQ_2024/GIS_Data/'
dfAll_orig = pd.read_csv(gis_path + "OEAT_All_WQ-2023-Jan-16.csv", low_memory=False)

In [3]:
# Calculate the year list.
unique_years = dfAll_orig['Year'].unique()
unique_years.sort()
print("Unique years in 'Year' column:", unique_years)

Unique years in 'Year' column: [2015 2016 2017 2018 2019 2020 2021 2022 2023]


In [4]:
area_shortnames = {
    'Guana Tolomato Matanzas': 'GTM',
    'Estero Bay': 'EB',
    'Charlotte Harbor': 'CH',
    'Biscayne Bay': 'BB',
    'Big Bend Seagrasses':'BBS'
}

param_shortnames = {
    'Salinity': 'Sal_ppt',
    'Total Nitrogen': 'TN_mgl',
    'Dissolved Oxygen': 'DO_mgl',
    'Turbidity':'Turb_ntu',
    'Secchi Depth':'Secc_m',
    'Water Temperature':'T_c'
}

# Set input parameters
waterbody_names = [
    'Guana Tolomato Matanzas',
    'Estero Bay',
    'Charlotte Harbor',
    'Biscayne Bay',
    'Big Bend Seagrasses'
]

parameter_names = ['Dissolved Oxygen', 'Salinity', 'Secchi Depth', 'Total Nitrogen', 'Turbidity', 'Water Temperature']
years = unique_years
seasons = ['Fall', 'Spring', 'Summer', 'Winter']
shp_folder = gis_path + r"shapefiles"

# 2. Generate Shapefiles<a class="anchor" id="create_shp"></a>
## First, you need to aggregate the repeatly observation data into one value in the time period.

In [5]:
dfMean = dfAll_orig.groupby(['WaterBody','ParameterName','Year','Season','Latitude_DD','Longitude_DD'])["ResultValue"].agg("mean").reset_index()    

## Second, cleanup the folder files

In [7]:
RK.delete_all_files(shp_folder)

## Third, generate the shapefiles of input points for each waterbody, parameter, year, and season

In [8]:
# Create shapefiles by waterbody,parameter，year, and season
# Print number of data points in each shapefile
RK.create_shp_season(dfMean, waterbody_names, parameter_names, years, seasons, shp_folder)

Number of data rows for GTM, Dissolved Oxygen, 2015, Fall: 8
Shapefile for GTM: DO_mgl for year 2015 and season Fall has been saved as SHP_GTM_DO_mgl_2015_Fall.shp
Number of data rows for GTM, Salinity, 2015, Fall: 8
Shapefile for GTM: Sal_ppt for year 2015 and season Fall has been saved as SHP_GTM_Sal_ppt_2015_Fall.shp
No data found for area: GTM, parameter: Secchi Depth, year: 2015, and season: Fall
No data found for area: GTM, parameter: Total Nitrogen, year: 2015, and season: Fall
Number of data rows for GTM, Turbidity, 2015, Fall: 4
Shapefile for GTM: Turb_ntu for year 2015 and season Fall has been saved as SHP_GTM_Turb_ntu_2015_Fall.shp
Number of data rows for GTM, Water Temperature, 2015, Fall: 8
Shapefile for GTM: T_c for year 2015 and season Fall has been saved as SHP_GTM_T_c_2015_Fall.shp
Number of data rows for GTM, Dissolved Oxygen, 2015, Spring: 8
Shapefile for GTM: DO_mgl for year 2015 and season Spring has been saved as SHP_GTM_DO_mgl_2015_Spring.shp
Number of data rows 

Shapefile for GTM: T_c for year 2017 and season Spring has been saved as SHP_GTM_T_c_2017_Spring.shp
Number of data rows for GTM, Dissolved Oxygen, 2017, Summer: 17
Shapefile for GTM: DO_mgl for year 2017 and season Summer has been saved as SHP_GTM_DO_mgl_2017_Summer.shp
Number of data rows for GTM, Salinity, 2017, Summer: 17
Shapefile for GTM: Sal_ppt for year 2017 and season Summer has been saved as SHP_GTM_Sal_ppt_2017_Summer.shp
Number of data rows for GTM, Secchi Depth, 2017, Summer: 9
Shapefile for GTM: Secc_m for year 2017 and season Summer has been saved as SHP_GTM_Secc_m_2017_Summer.shp
Number of data rows for GTM, Total Nitrogen, 2017, Summer: 14
Shapefile for GTM: TN_mgl for year 2017 and season Summer has been saved as SHP_GTM_TN_mgl_2017_Summer.shp
Number of data rows for GTM, Turbidity, 2017, Summer: 4
Shapefile for GTM: Turb_ntu for year 2017 and season Summer has been saved as SHP_GTM_Turb_ntu_2017_Summer.shp
Number of data rows for GTM, Water Temperature, 2017, Summer:

Shapefile for GTM: Secc_m for year 2019 and season Summer has been saved as SHP_GTM_Secc_m_2019_Summer.shp
Number of data rows for GTM, Total Nitrogen, 2019, Summer: 22
Shapefile for GTM: TN_mgl for year 2019 and season Summer has been saved as SHP_GTM_TN_mgl_2019_Summer.shp
Number of data rows for GTM, Turbidity, 2019, Summer: 15
Shapefile for GTM: Turb_ntu for year 2019 and season Summer has been saved as SHP_GTM_Turb_ntu_2019_Summer.shp
Number of data rows for GTM, Water Temperature, 2019, Summer: 15
Shapefile for GTM: T_c for year 2019 and season Summer has been saved as SHP_GTM_T_c_2019_Summer.shp
Number of data rows for GTM, Dissolved Oxygen, 2019, Winter: 11
Shapefile for GTM: DO_mgl for year 2019 and season Winter has been saved as SHP_GTM_DO_mgl_2019_Winter.shp
Number of data rows for GTM, Salinity, 2019, Winter: 12
Shapefile for GTM: Sal_ppt for year 2019 and season Winter has been saved as SHP_GTM_Sal_ppt_2019_Winter.shp
Number of data rows for GTM, Secchi Depth, 2019, Winte

Shapefile for GTM: Turb_ntu for year 2021 and season Summer has been saved as SHP_GTM_Turb_ntu_2021_Summer.shp
Number of data rows for GTM, Water Temperature, 2021, Summer: 27
Shapefile for GTM: T_c for year 2021 and season Summer has been saved as SHP_GTM_T_c_2021_Summer.shp
Number of data rows for GTM, Dissolved Oxygen, 2021, Winter: 4
Shapefile for GTM: DO_mgl for year 2021 and season Winter has been saved as SHP_GTM_DO_mgl_2021_Winter.shp
Number of data rows for GTM, Salinity, 2021, Winter: 5
Shapefile for GTM: Sal_ppt for year 2021 and season Winter has been saved as SHP_GTM_Sal_ppt_2021_Winter.shp
No data found for area: GTM, parameter: Secchi Depth, year: 2021, and season: Winter
No data found for area: GTM, parameter: Total Nitrogen, year: 2021, and season: Winter
Number of data rows for GTM, Turbidity, 2021, Winter: 4
Shapefile for GTM: Turb_ntu for year 2021 and season Winter has been saved as SHP_GTM_Turb_ntu_2021_Winter.shp
Number of data rows for GTM, Water Temperature, 20

Shapefile for EB: Sal_ppt for year 2015 and season Fall has been saved as SHP_EB_Sal_ppt_2015_Fall.shp
No data found for area: EB, parameter: Secchi Depth, year: 2015, and season: Fall
No data found for area: EB, parameter: Total Nitrogen, year: 2015, and season: Fall
Number of data rows for EB, Turbidity, 2015, Fall: 2
Shapefile for EB: Turb_ntu for year 2015 and season Fall has been saved as SHP_EB_Turb_ntu_2015_Fall.shp
Number of data rows for EB, Water Temperature, 2015, Fall: 3
Shapefile for EB: T_c for year 2015 and season Fall has been saved as SHP_EB_T_c_2015_Fall.shp
Number of data rows for EB, Dissolved Oxygen, 2015, Spring: 3
Shapefile for EB: DO_mgl for year 2015 and season Spring has been saved as SHP_EB_DO_mgl_2015_Spring.shp
Number of data rows for EB, Salinity, 2015, Spring: 3
Shapefile for EB: Sal_ppt for year 2015 and season Spring has been saved as SHP_EB_Sal_ppt_2015_Spring.shp
No data found for area: EB, parameter: Secchi Depth, year: 2015, and season: Spring
No da

...

Shapefile for BB: T_c for year 2022 and season Winter has been saved as SHP_BB_T_c_2022_Winter.shp
No data found for area: BB, parameter: Dissolved Oxygen, year: 2023, and season: Fall
No data found for area: BB, parameter: Salinity, year: 2023, and season: Fall
No data found for area: BB, parameter: Secchi Depth, year: 2023, and season: Fall
No data found for area: BB, parameter: Total Nitrogen, year: 2023, and season: Fall
No data found for area: BB, parameter: Turbidity, year: 2023, and season: Fall
No data found for area: BB, parameter: Water Temperature, year: 2023, and season: Fall
Number of data rows for BB, Dissolved Oxygen, 2023, Spring: 58
Shapefile for BB: DO_mgl for year 2023 and season Spring has been saved as SHP_BB_DO_mgl_2023_Spring.shp
Number of data rows for BB, Salinity, 2023, Spring: 52
Shapefile for BB: Sal_ppt for year 2023 and season Spring has been saved as SHP_BB_Sal_ppt_2023_Spring.shp
No data found for area: BB, parameter: Secchi Depth, year: 2023, and season

Shapefile for BBS: Sal_ppt for year 2016 and season Summer has been saved as SHP_BBS_Sal_ppt_2016_Summer.shp
Number of data rows for BBS, Secchi Depth, 2016, Summer: 1
Shapefile for BBS: Secc_m for year 2016 and season Summer has been saved as SHP_BBS_Secc_m_2016_Summer.shp
Number of data rows for BBS, Total Nitrogen, 2016, Summer: 1
Shapefile for BBS: TN_mgl for year 2016 and season Summer has been saved as SHP_BBS_TN_mgl_2016_Summer.shp
Number of data rows for BBS, Turbidity, 2016, Summer: 2
Shapefile for BBS: Turb_ntu for year 2016 and season Summer has been saved as SHP_BBS_Turb_ntu_2016_Summer.shp
Number of data rows for BBS, Water Temperature, 2016, Summer: 2
Shapefile for BBS: T_c for year 2016 and season Summer has been saved as SHP_BBS_T_c_2016_Summer.shp
Number of data rows for BBS, Dissolved Oxygen, 2016, Winter: 2
Shapefile for BBS: DO_mgl for year 2016 and season Winter has been saved as SHP_BBS_DO_mgl_2016_Winter.shp
Number of data rows for BBS, Salinity, 2016, Winter: 2


Shapefile for BBS: Turb_ntu for year 2018 and season Summer has been saved as SHP_BBS_Turb_ntu_2018_Summer.shp
Number of data rows for BBS, Water Temperature, 2018, Summer: 43
Shapefile for BBS: T_c for year 2018 and season Summer has been saved as SHP_BBS_T_c_2018_Summer.shp
Number of data rows for BBS, Dissolved Oxygen, 2018, Winter: 44
Shapefile for BBS: DO_mgl for year 2018 and season Winter has been saved as SHP_BBS_DO_mgl_2018_Winter.shp
Number of data rows for BBS, Salinity, 2018, Winter: 45
Shapefile for BBS: Sal_ppt for year 2018 and season Winter has been saved as SHP_BBS_Sal_ppt_2018_Winter.shp
Number of data rows for BBS, Secchi Depth, 2018, Winter: 6
Shapefile for BBS: Secc_m for year 2018 and season Winter has been saved as SHP_BBS_Secc_m_2018_Winter.shp
Number of data rows for BBS, Total Nitrogen, 2018, Winter: 44
Shapefile for BBS: TN_mgl for year 2018 and season Winter has been saved as SHP_BBS_TN_mgl_2018_Winter.shp
Number of data rows for BBS, Turbidity, 2018, Winter

Shapefile for BBS: DO_mgl for year 2020 and season Winter has been saved as SHP_BBS_DO_mgl_2020_Winter.shp
Number of data rows for BBS, Salinity, 2020, Winter: 36
Shapefile for BBS: Sal_ppt for year 2020 and season Winter has been saved as SHP_BBS_Sal_ppt_2020_Winter.shp
Number of data rows for BBS, Secchi Depth, 2020, Winter: 32
Shapefile for BBS: Secc_m for year 2020 and season Winter has been saved as SHP_BBS_Secc_m_2020_Winter.shp
Number of data rows for BBS, Total Nitrogen, 2020, Winter: 34
Shapefile for BBS: TN_mgl for year 2020 and season Winter has been saved as SHP_BBS_TN_mgl_2020_Winter.shp
Number of data rows for BBS, Turbidity, 2020, Winter: 39
Shapefile for BBS: Turb_ntu for year 2020 and season Winter has been saved as SHP_BBS_Turb_ntu_2020_Winter.shp
Number of data rows for BBS, Water Temperature, 2020, Winter: 38
Shapefile for BBS: T_c for year 2020 and season Winter has been saved as SHP_BBS_T_c_2020_Winter.shp
Number of data rows for BBS, Dissolved Oxygen, 2021, Fall:

Shapefile for BBS: Secc_m for year 2022 and season Winter has been saved as SHP_BBS_Secc_m_2022_Winter.shp
Number of data rows for BBS, Total Nitrogen, 2022, Winter: 30
Shapefile for BBS: TN_mgl for year 2022 and season Winter has been saved as SHP_BBS_TN_mgl_2022_Winter.shp
Number of data rows for BBS, Turbidity, 2022, Winter: 36
Shapefile for BBS: Turb_ntu for year 2022 and season Winter has been saved as SHP_BBS_Turb_ntu_2022_Winter.shp
Number of data rows for BBS, Water Temperature, 2022, Winter: 37
Shapefile for BBS: T_c for year 2022 and season Winter has been saved as SHP_BBS_T_c_2022_Winter.shp
Number of data rows for BBS, Dissolved Oxygen, 2023, Fall: 4
Shapefile for BBS: DO_mgl for year 2023 and season Fall has been saved as SHP_BBS_DO_mgl_2023_Fall.shp
No data found for area: BBS, parameter: Salinity, year: 2023, and season: Fall
No data found for area: BBS, parameter: Secchi Depth, year: 2023, and season: Fall
Number of data rows for BBS, Total Nitrogen, 2023, Fall: 4
Shape

# 3. Regression Kriging<a class="anchor" id="rk"></a>

## Loop for all parameters


In [6]:
season_result = pd.read_csv(gis_path+"Season_results.csv")

In [11]:
# If you would like to partially generate results in some rows, you could use range(a,b):
# If you would like to generate all rows automated, you could use season_result.index:

# for i in season_result.index:
for i in range(15,240):
    process,rmse,me,count = RK.rk_interpolation(method = "rk",
                                       folder_path = gis_path,
                                       waterbody = season_result.iloc[i]["WaterBodyA"],
                                       parameter = season_result.iloc[i]["ParameterA"],
                                       year      = season_result.iloc[i]["Year"],
                                       season    = season_result.iloc[i]['Season'],
                                       covariates= season_result.iloc[i]["Covariates"])
    print(process,rmse,me,count)
    season_result['Process'][i:i+1] = process
    season_result['RMSE'][i:i+1] = rmse
    season_result['ME'][i:i+1] = me
    season_result['Count'][i:i+1] = count
season_result.to_csv(gis_path+"Season_results.csv")

Processing file: SHP_EB_TN_mgl_2018_Spring.shp
--- Time lapse: 41.65754580497742 seconds ---
1 0.10658072725 0.00141980489225 27
No data for RK interpolation in SHP_CH_TN_mgl_2016_Summer.shp, skipping
0 nan nan 0
No data for RK interpolation in SHP_CH_TN_mgl_2016_Fall.shp, skipping
0 nan nan 0
TN_mgl in 2016 Winter caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 9
No data for RK interpolation in SHP_CH_TN_mgl_2017_Spring.shp, skipping
0 nan nan 0
TN_mgl in 2017 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 1
TN_mgl in 2017 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 2
TN_mgl in 2017 Winter caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 1
Processing file: SHP_CH_TN_mgl_2018_Spr

Processing file: SHP_BBS_Sal_ppt_2021_Summer.shp
--- Time lapse: 477.1718521118164 seconds ---
1 6.40175754484 -0.911584009012 36
Processing file: SHP_BBS_Sal_ppt_2021_Fall.shp
--- Time lapse: 434.0045585632324 seconds ---
1 -1.79769313486e+308 -1.79769313486e+308 30
Processing file: SHP_BBS_Sal_ppt_2021_Winter.shp
--- Time lapse: 458.559903383255 seconds ---
1 3.51081782338 -0.553751687697 33
Processing file: SHP_BBS_Sal_ppt_2022_Spring.shp
--- Time lapse: 259.6676445007324 seconds ---
1 -1.79769313486e+308 -1.79769313486e+308 17
DO_mgl in 2015 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 8
DO_mgl in 2015 Winter caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 8
DO_mgl in 2016 Spring caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 8
DO_mgl in 2016 Summer cause

Processing file: SHP_EB_Turb_ntu_2018_Spring.shp
--- Time lapse: 39.23244023323059 seconds ---
1 1.62309610261 0.033657733306 30
Turb_ntu in 2016 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 3
Turb_ntu in 2016 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 3
Processing file: SHP_CH_Turb_ntu_2016_Winter.shp
--- Time lapse: 138.96203517913818 seconds ---
1 1.03899028903 -0.0667798356833 12
Turb_ntu in 2017 Spring caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 3
Turb_ntu in 2017 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 4
Turb_ntu in 2017 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 5
Tur

T_c in 2016 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 8
T_c in 2016 Winter caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 8
T_c in 2017 Spring caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 8
Processing file: SHP_GTM_T_c_2017_Summer.shp
--- Time lapse: 41.00446534156799 seconds ---
1 0.545367873869 0.0108285868841 18
T_c in 2016 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 3
T_c in 2016 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 3
T_c in 2016 Winter caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 nan nan 3
T_c in 2017 Spring 